In [1]:
from src.llm_plan.llm import GPT_OSS

system_prompt = "You are a useful assistant."
prompt = "Tell me a joke."

model = GPT_OSS(map="cpu", reasoning="Low")

model.generate_sync(system_prompt, prompt)

/home/emanuele/MultiAgentPlanning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
MXFP4 quantization requires triton >= 3.4.0 and kernels installed, we will default to dequantizing the model to bf16
Loading checkpoint shards: 100%|██████████| 15/15 [04:46<00:00, 19.13s/it]


In [ ]:
prompt = "What is the capital of France?"
model.generate_sync(system_prompt, prompt)